In [2]:
import os
import pandas
from wq_gps_match import timestamp_match

In [3]:
# set wd to Arcproject-wq-processing folder
wd = os.path.abspath(os.path.join(os.path.dirname(os.path.dirname("__file__"))))

# example shp
shp = os.path.join(wd, "wq_gps_match", "examples", "Arc_040413_GPS", "040413_PosnPnt.shp")

# example water quality
wq_file = os.path.join(wd, "wq_gps_match", "examples", "Arc_040413_WQ", "Arc_040413_wqt_cc.csv")

print(shp)
print(wq_file)

C:\Users\Andy\Documents\arcproject-wq-processing\wq_gps_match\examples\Arc_040413_GPS\040413_PosnPnt.shp
C:\Users\Andy\Documents\arcproject-wq-processing\wq_gps_match\examples\Arc_040413_WQ\Arc_040413_wqt_cc.csv


Import CSV file from the MiniSonde as Pandas Dataframe

In [4]:
water_quality = timestamp_match.wq_from_csv(wq_file)
print(water_quality)

              Date_Time   Temp    pH SpCond   Sal DEP25  PAR RPAR TurbSC  \
1   2013-04-04 13:01:03  17.40  7.84    520  0.26  0.39  573  417   13.4   
2   2013-04-04 13:01:04  17.40  7.84    520  0.26  0.35  573  417   13.4   
3   2013-04-04 13:01:05  17.40  7.88    520  0.26  0.35  576  419   13.5   
4   2013-04-04 13:01:06  17.41  7.85    521  0.26  0.39  576  417   13.5   
5   2013-04-04 13:01:07  17.41  7.85    521  0.26  0.39  576  417   13.5   
6   2013-04-04 13:01:08  17.40  7.84    521  0.26  0.37  576  415   13.4   
7   2013-04-04 13:01:09  17.40  7.84    521  0.26  0.37  576  415   13.4   
8   2013-04-04 13:01:10  17.39  7.83    520  0.26  0.35  573  412   13.4   
9   2013-04-04 13:01:11  17.39  7.83    520  0.26  0.35  573  412   13.4   
10  2013-04-04 13:01:12  17.38  7.83    521  0.26  0.38  569  407   13.4   
11  2013-04-04 13:01:13  17.39  7.83    521  0.26  0.38  569  407   13.4   
12  2013-04-04 13:01:14  17.39  7.84    521  0.26  0.38  566  408   13.4   
13  2013-04-

In [8]:
shp_df = timestamp_match.shp2dataframe(shp)

print(shp_df)

                     Date                               XY
0     2013-04-04 08:18:47  [-121.675981849, 38.2332247674]
1     2013-04-04 08:18:48  [-121.675986001, 38.2332196115]
2     2013-04-04 08:18:49   [-121.675990421, 38.233214315]
3     2013-04-04 08:18:50   [-121.675994058, 38.233209039]
4     2013-04-04 08:18:51  [-121.675997428, 38.2332039378]
5     2013-04-04 08:18:52  [-121.676000626, 38.2331986402]
6     2013-04-04 08:18:53   [-121.67600359, 38.2331933725]
7     2013-04-04 08:18:54   [-121.67600635, 38.2331880878]
8     2013-04-04 08:18:55  [-121.676008961, 38.2331826173]
9     2013-04-04 08:18:56  [-121.676011313, 38.2331771616]
10    2013-04-04 08:18:57  [-121.676013455, 38.2331715947]
11    2013-04-04 08:18:58  [-121.676015148, 38.2331660353]
12    2013-04-04 08:18:59  [-121.676016576, 38.2331604596]
13    2013-04-04 08:19:00  [-121.676017562, 38.2331550216]
14    2013-04-04 08:19:01   [-121.67601802, 38.2331496179]
15    2013-04-04 08:19:02  [-121.676018196, 38.233144342

In [6]:
merged = timestamp_match.JoinByTimeStamp(water_quality, shp_df)
#print(merged)

matches = merged[0]

print('Dimensions - origninal WQ:  {},  Matches: {}'.format( water_quality.shape, matches.shape))


def nrows(df):
    n= df.shape[0]
    return n

print('Number of rows: original = {}, new = {}'.format(nrows(water_quality), nrows(matches)))

diff = nrows(water_quality) - nrows(matches)
print('{} unmatched rows in the water quality dataset'.format(diff))

percent_match = float(nrows(matches))/float(nrows(water_quality)) * 100
print(nrows(water_quality))

print('Percent Match: {} %'.format(percent_match))

Dimensions - origninal WQ:  (977, 11),  Matches: (0, 13)
Number of rows: original = 977, new = 0
977 unmatched rows in the water quality dataset
977
Percent Match: 0.0 %


In [7]:
merged[1]

,Date_Time,Temp,pH,SpCond,Sal,DEP25,PAR,RPAR,TurbSC,CHL,CHL.1,Date,XY
0,2013-04-04 13:01:03,17.40,7.84,520,0.26,0.39,573,417,13.4,5.34,0.05078,NaT,NaN
1,2013-04-04 13:01:04,17.40,7.84,520,0.26,0.35,573,417,13.4,5.34,0.05078,NaT,NaN
2,2013-04-04 13:01:05,17.40,7.88,520,0.26,0.35,576,419,13.5,5.29,0.05095,NaT,NaN
3,2013-04-04 13:01:06,17.41,7.85,521,0.26,0.39,576,417,13.5,5.31,0.05073,NaT,NaN
4,2013-04-04 13:01:07,17.41,7.85,521,0.26,0.39,576,417,13.5,5.31,0.05073,NaT,NaN
5,2013-04-04 13:01:08,17.40,7.84,521,0.26,0.37,576,415,13.4,5.28,0.05064,NaT,NaN
6,2013-04-04 13:01:09,17.40,7.84,521,0.26,0.37,576,415,13.4,5.28,0.05064,NaT,NaN
7,2013-04-04 13:01:10,17.39,7.83,520,0.26,0.35,573,412,13.4,5.27,0.05036,NaT,NaN
8,2013-04-04 13:01:11,17.39,7.83,520,0.26,0.35,573,412,13.4,5.27,0.05036,NaT,NaN
9,2013-04-04 13:01:12,17.38,7.83,521,0.26,0.38,569,407,13.4,5.25,0.05021,NaT,NaN


In [8]:
# export matchs to shapefile?

timestamp_match.write_shp("test.shp", matches)